## Import libraries

In [2]:
import pandas as pd
import numpy as np
import os
import random
from pdb import set_trace as bp
import matplotlib.pyplot as plt
from collections import defaultdict
import pickle
from sklearn.utils import shuffle
from dataGenerator import DataGenerator
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import text_to_word_sequence
from utils import plotGraph,plotLengthHistogram,plotHist,labelPlot

Using TensorFlow backend.


In [3]:
dataGenerator = DataGenerator(batchSize = 50, questionLen = 25)
trainX, trainY = dataGenerator.getTrainBatch()
print trainX.shape
print trainY.shape

validX, validY = dataGenerator.getValidBatch()
print validX.shape
print validY.shape

(2, 50, 25, 100)
(50,)
(2, 50, 25, 100)
(50,)


In [4]:
%%time
for i in range(1000):
    trainX, trainY = dataGenerator.getTrainBatch()

CPU times: user 9.16 s, sys: 0 ns, total: 9.16 s
Wall time: 9.16 s


In [5]:
%%time
for i in range(1000):
    validX, validY = dataGenerator.getValidBatch()

CPU times: user 9.54 s, sys: 0 ns, total: 9.54 s
Wall time: 9.55 s
